In [6]:
import os
import sqlalchemy as sq

# temporary file for sql database
if os.path.exists("file.db"):
    os.remove("file.db")

In [7]:
# import prices from csv file
import pandas as pd
prices = pd.read_csv("data/price.csv", index_col=0, header=0, parse_dates=True)

In [8]:
from pyutil.sql.interfaces.symbols.symbol import Symbol, SymbolType
# Here you can go wild and point to an existing MongoDB Server. I use an inmemory-solution here...
from mongomock import MongoClient

# point to an inmemory Mongo Database
Symbol.mongo_database = MongoClient()["test"]

In [9]:
from pyutil.sql.base import Base
from pyutil.sql.session import session 


with session('sqlite:///file.db', echo=False, base=Base) as s:
    for key, data in prices.items():
        symbol = Symbol(name=key, group=SymbolType.equities, internal=key)
        # upsert the prices
        symbol.series["PX_LAST"] = data.dropna()
        s.add(symbol)
    s.commit()  
    # this will raise an error as a Symbol with the same name has already been defined
    # same issue if we execute this cell a second time...
    # you may need to delete file.db by hand
    # symbol = Symbol(name="A", group=SymbolType.equities, internal="Peter Maffay")
    # session.add(symbol)
    
    # commit is called whenever we leave the scope, so no need to do this explicitly here
    # s.commit()


In [10]:
with session('sqlite:///file.db', echo=False, base=Base) as s:
    for symbol in s.query(Symbol):
        print(symbol)
        print(symbol.series["PX_LAST"].tail(2))

A
2015-04-21    1202.34
2015-04-22    1200.59
Name: A, dtype: float64
B
2015-04-21    27850.49
2015-04-22    27964.84
Name: B, dtype: float64
C
2015-04-17    53954.79
2015-04-20    53761.27
Name: C, dtype: float64
D
2015-04-21    2651.41
2015-04-22    2638.95
Name: D, dtype: float64
E
2015-04-20    2100.40
2015-04-21    2097.29
Name: E, dtype: float64
F
2015-04-21    3719.38
2015-04-22    3742.77
Name: F, dtype: float64
G
2015-04-20    3059.73
2015-04-21    3065.53
Name: G, dtype: float64


In [14]:
with session('sqlite:///file.db', echo=False, base=Base) as s:
    a = s.query(Symbol).filter(Symbol.name=="A").one()
    b = s.query(Symbol).filter(Symbol.name=="B").one()
    symbols = [a,b]
    t = Symbol.reference_frame(symbols)
    print(t)
    h = Symbol.pandas_frame(key="PX_LAST", products=symbols).tail(5)
    print(h)
    



          Sector Internal
symbol                   
A       Equities        A
B       Equities        B
symbol            A         B
2015-04-16  1198.56  27739.71
2015-04-17  1204.27  27653.12
2015-04-20  1195.88  27094.93
2015-04-21  1202.34  27850.49
2015-04-22  1200.59  27964.84
